# Channel Head Coupling Analysis - Experiment Template

This notebook provides a clean template for running analysis experiments with different parameters.

**Key features:**
- All experiment parameters defined in one cell
- Outputs automatically organized by experiment name/threshold
- Easy to duplicate for new experiments

## How to use:
1. **Duplicate this notebook** for each experiment (e.g., `experiment_th500.ipynb`)
2. **Modify the parameters** in the "Experiment Configuration" cell below
3. **Run all cells** to execute the analysis
4. Results are saved to `data/outputs/experiments/{EXPERIMENT_NAME}/`

---
## Experiment Configuration

**Modify these parameters for your experiment:**

In [ ]:
# ============================================================================
# EXPERIMENT PARAMETERS - MODIFY THESE FOR YOUR EXPERIMENT
# ============================================================================

# Experiment identification
EXPERIMENT_NAME = "th500_test"  # Unique name for this experiment
EXPERIMENT_NOTES = "Testing higher stream threshold (500 vs default 145)"

# Stream extraction threshold (default in paper: 145)
# Higher = fewer, larger streams; Lower = more, smaller streams
STREAM_THRESHOLD = 500

# Basins to analyze (set to None for all 18 basins)
# Examples:
#   BASINS_TO_RUN = None  # All 18 basins
#   BASINS_TO_RUN = ["inyo", "humboldt"]  # Just these two
#   BASINS_TO_RUN = ["inyo"]  # Single basin for quick test
BASINS_TO_RUN = ["inyo", "humboldt", "calnalpine"]  # Quick test set

# Limit outlets per basin (None = all outlets, useful for quick tests)
MAX_OUTLETS_PER_BASIN = None

# Connectivity for coupling detection (4 or 8)
# 8 = include diagonal neighbors (more sensitive)
# 4 = only cardinal directions (more conservative)
CONNECTIVITY = 8

# ============================================================================
print(f"Experiment: {EXPERIMENT_NAME}")
print(f"Notes: {EXPERIMENT_NOTES}")
print(f"Stream threshold: {STREAM_THRESHOLD}")
print(f"Basins: {BASINS_TO_RUN or 'All 18'}")
print(f"Max outlets per basin: {MAX_OUTLETS_PER_BASIN or 'All'}")
print(f"Connectivity: {CONNECTIVITY}")

---
## Setup and Imports

In [ ]:
import sys
import pathlib
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import topotoolbox as tt3

# Add project root so "channel_heads" package is visible
project_root = pathlib.Path("/Users/guypi/Projects/channel-heads")
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Core analysis imports
from channel_heads import (
    CouplingAnalyzer,
    first_meet_pairs_for_outlet,
    outlet_node_ids_from_streampoi,
    # Lengthwise asymmetry
    LengthwiseAsymmetryAnalyzer,
    compute_asymmetry_statistics,
    merge_coupling_and_asymmetry,
    # Basin configuration
    get_z_th,
    get_basin_config,
    get_reference_delta_L,
    BASIN_CONFIG,
    # Path management
    CROPPED_DEMS_DIR,
    get_experiment_output_dir,
)

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)
%matplotlib inline

# Create output directory for this experiment
OUTPUT_DIR = get_experiment_output_dir(EXPERIMENT_NAME)
print(f"\nOutput directory: {OUTPUT_DIR}")
print("Setup complete!")

---
## Basin Configuration

In [ ]:
# All 18 basins from Goren & Shelef (2024)
ALL_BASINS = {
    "inyo": {"dem_path": CROPPED_DEMS_DIR / "Inyo_strm_crop.tif", "paper_name": "inyo"},
    "humboldt": {"dem_path": CROPPED_DEMS_DIR / "Humboldt_strm_crop.tif", "paper_name": "humboldt"},
    "calnalpine": {"dem_path": CROPPED_DEMS_DIR / "CalnAlpine_strm_crop.tif", "paper_name": "clanalpine"},
    "daqing": {"dem_path": CROPPED_DEMS_DIR / "Daqing_strm_crop.tif", "paper_name": "daqing"},
    "kammanasie": {"dem_path": CROPPED_DEMS_DIR / "Kammanasie_strm_crop.tif", "paper_name": "kammanassie"},
    "luliang": {"dem_path": CROPPED_DEMS_DIR / "Luliang_strm_crop.tif", "paper_name": "luliang"},
    "finisterre": {"dem_path": CROPPED_DEMS_DIR / "Finisterre_strm_crop.tif", "paper_name": "finisterre"},
    "taiwan": {"dem_path": CROPPED_DEMS_DIR / "Taiwan_strm_crop.tif", "paper_name": "taiwan"},
    "panamint": {"dem_path": CROPPED_DEMS_DIR / "Panamint_strm_crop.tif", "paper_name": "panamint"},
    "sakhalin": {"dem_path": CROPPED_DEMS_DIR / "Sakhalin_strm_crop.tif", "paper_name": "sakhalin"},
    "vallefertil": {"dem_path": CROPPED_DEMS_DIR / "ValleFertil_strm_crop.tif", "paper_name": "vallefertil"},
    "sierramadre": {"dem_path": CROPPED_DEMS_DIR / "SierraMadre_strm_crop.tif", "paper_name": "sierramadre"},
    "sierranevadaspain": {"dem_path": CROPPED_DEMS_DIR / "SierraNevadaSpain_strm_crop.tif", "paper_name": "sierranevada_spain"},
    "piedepalo": {"dem_path": CROPPED_DEMS_DIR / "PieDePalo_strm_crop.tif", "paper_name": "piedepalo"},
    "toano": {"dem_path": CROPPED_DEMS_DIR / "Toano_strm_crop.tif", "paper_name": "toano"},
    "troodos": {"dem_path": CROPPED_DEMS_DIR / "Troodos_strm_crop.tif", "paper_name": "troodos"},
    "tsugaru": {"dem_path": CROPPED_DEMS_DIR / "Tsugaru_strm_crop.tif", "paper_name": "tsugaru"},
    "yoro": {"dem_path": CROPPED_DEMS_DIR / "Yoro_strm_crop.tif", "paper_name": "yoro"},
}

# Filter to selected basins
if BASINS_TO_RUN is not None:
    BASINS_TO_ANALYZE = {k: v for k, v in ALL_BASINS.items() if k in BASINS_TO_RUN}
else:
    BASINS_TO_ANALYZE = ALL_BASINS

# Show basin configurations
print("=" * 80)
print(f"Experiment: {EXPERIMENT_NAME}")
print(f"Stream threshold: {STREAM_THRESHOLD}")
print(f"Basins to analyze: {len(BASINS_TO_ANALYZE)}")
print("=" * 80)

for name, config in BASINS_TO_ANALYZE.items():
    basin_config = get_basin_config(config["paper_name"])
    dem_exists = config['dem_path'].exists()
    status = "OK" if dem_exists else "MISSING"
    print(f"  {basin_config['full_name'][:40]:<40} z_th={basin_config['z_th']:>4}m [{status}]")

---
## Analysis Functions

In [ ]:
def load_and_prepare_basin(basin_name, dem_path, z_th, stream_threshold):
    """Load DEM, apply z_th mask, and create stream network."""
    print(f"Loading {basin_name}...")
    
    # Load DEM
    dem = tt3.read_tif(str(dem_path))
    print(f"  DEM shape: {dem.z.shape}")
    print(f"  Elevation range: {np.nanmin(dem.z):.0f} - {np.nanmax(dem.z):.0f} m")
    
    # Apply z_th mask
    dem.z[dem.z < z_th] = np.nan
    valid_pct = 100 * np.sum(~np.isnan(dem.z)) / dem.z.size
    print(f"  Applied z_th={z_th} m mask: {valid_pct:.1f}% valid pixels")
    
    # Create flow and stream objects
    fd = tt3.FlowObject(dem)
    s = tt3.StreamObject(fd, threshold=stream_threshold)
    
    # Count network features
    n_outlets = np.sum(s.streampoi('outlets'))
    n_heads = np.sum(s.streampoi('channelheads'))
    n_confluences = np.sum(s.streampoi('confluences'))
    print(f"  Stream network (threshold={stream_threshold}):")
    print(f"    Outlets: {n_outlets}, Heads: {n_heads}, Confluences: {n_confluences}")
    
    return {"dem": dem, "fd": fd, "s": s}


def run_full_analysis(s, fd, dem, lat, connectivity, max_outlets=None):
    """Run coupling + lengthwise asymmetry analysis for all outlets."""
    outlets = outlet_node_ids_from_streampoi(s)
    outlets = [int(o) for o in outlets]
    
    if max_outlets and len(outlets) > max_outlets:
        print(f"  Limiting to first {max_outlets} outlets (of {len(outlets)})")
        outlets = outlets[:max_outlets]
    
    # Create analyzers
    coupling_an = CouplingAnalyzer(fd, s, dem, connectivity=connectivity)
    asymmetry_an = LengthwiseAsymmetryAnalyzer(s, dem, lat=lat)
    
    coupling_dfs = []
    asymmetry_dfs = []
    
    for idx, outlet in enumerate(outlets, 1):
        print(f"  [{idx}/{len(outlets)}] outlet={outlet}", end="", flush=True)
        
        pairs_at_confluence, basin_heads = first_meet_pairs_for_outlet(s, outlet)
        
        if not pairs_at_confluence:
            print(" (no pairs)")
            continue
        
        # Clear caches between outlets
        coupling_an.clear_cache()
        asymmetry_an.clear_cache()
        
        df_coupling = coupling_an.evaluate_pairs_for_outlet(outlet, pairs_at_confluence)
        df_asymmetry = asymmetry_an.evaluate_pairs_for_outlet(outlet, pairs_at_confluence)
        
        n_pairs = len(df_coupling)
        n_touching = df_coupling['touching'].sum() if not df_coupling.empty else 0
        print(f" ({n_pairs} pairs, {n_touching} touching)")
        
        if not df_coupling.empty:
            coupling_dfs.append(df_coupling)
        if not df_asymmetry.empty:
            asymmetry_dfs.append(df_asymmetry)
    
    if not coupling_dfs:
        return pd.DataFrame()
    
    df_coupling_all = pd.concat(coupling_dfs, ignore_index=True)
    df_asymmetry_all = pd.concat(asymmetry_dfs, ignore_index=True)
    df_combined = merge_coupling_and_asymmetry(df_coupling_all, df_asymmetry_all)
    df_combined.sort_values(["outlet", "confluence", "head_1", "head_2"], inplace=True, ignore_index=True)
    
    return df_combined

---
## Run Analysis

In [ ]:
# Store results
basin_results = {}
basin_data = {}

# Save experiment metadata
metadata = {
    "experiment_name": EXPERIMENT_NAME,
    "experiment_notes": EXPERIMENT_NOTES,
    "stream_threshold": STREAM_THRESHOLD,
    "connectivity": CONNECTIVITY,
    "max_outlets_per_basin": MAX_OUTLETS_PER_BASIN,
    "basins": list(BASINS_TO_ANALYZE.keys()),
    "timestamp": datetime.now().isoformat(),
}

import json
with open(OUTPUT_DIR / "experiment_metadata.json", "w") as f:
    json.dump(metadata, f, indent=2)
print(f"Saved experiment metadata to: {OUTPUT_DIR / 'experiment_metadata.json'}")

print("\n" + "=" * 80)

for basin_name, config in BASINS_TO_ANALYZE.items():
    if not config["dem_path"].exists():
        print(f"\nSkipping {basin_name}: DEM not found")
        continue
    
    print("\n" + "=" * 80)
    print(f"Processing: {basin_name.upper()}")
    print("=" * 80)
    
    basin_config = get_basin_config(config["paper_name"])
    z_th = basin_config["z_th"]
    lat = basin_config["lat"]
    
    # Load and prepare
    data = load_and_prepare_basin(
        basin_name=basin_name,
        dem_path=config["dem_path"],
        z_th=z_th,
        stream_threshold=STREAM_THRESHOLD,
    )
    basin_data[basin_name] = data
    
    # Run analysis
    print(f"\nRunning analysis (connectivity={CONNECTIVITY})...")
    df_results = run_full_analysis(
        s=data["s"],
        fd=data["fd"],
        dem=data["dem"],
        lat=lat,
        connectivity=CONNECTIVITY,
        max_outlets=MAX_OUTLETS_PER_BASIN,
    )
    basin_results[basin_name] = df_results
    
    # Save per-basin results
    output_path = OUTPUT_DIR / f"{basin_name}_results.csv"
    df_results.to_csv(output_path, index=False)
    print(f"Saved: {output_path}")

print("\n" + "=" * 80)
print(f"Analysis complete! Results in: {OUTPUT_DIR}")
print("=" * 80)

---
## Results Summary

In [ ]:
# Summary table
summary_rows = []

for basin_name, df in basin_results.items():
    if df.empty:
        continue
    
    config = get_basin_config(BASINS_TO_ANALYZE[basin_name]["paper_name"])
    ref = get_reference_delta_L(BASINS_TO_ANALYZE[basin_name]["paper_name"])
    
    n_pairs = len(df)
    n_touching = df['touching'].sum()
    touch_pct = 100 * n_touching / n_pairs if n_pairs > 0 else 0
    delta_L_stats = compute_asymmetry_statistics(df['delta_L'].dropna())
    
    summary_rows.append({
        "Basin": config["full_name"].split(',')[0],
        "z_th (m)": config["z_th"],
        "Total Pairs": n_pairs,
        "Touching": n_touching,
        "Touch %": f"{touch_pct:.1f}%",
        "delta_L (median)": f"{delta_L_stats['median']:.3f}",
        "Reference delta_L": f"{ref['median']:.2f}",
    })

df_summary = pd.DataFrame(summary_rows)
print(f"\n=== Experiment: {EXPERIMENT_NAME} ===")
print(f"Stream threshold: {STREAM_THRESHOLD}")
print()
print(df_summary.to_string(index=False))

# Save summary
df_summary.to_csv(OUTPUT_DIR / "summary.csv", index=False)
print(f"\nSaved summary to: {OUTPUT_DIR / 'summary.csv'}")

---
## Delta-L Distribution

In [ ]:
# Plot delta_L distribution for analyzed basins
n_basins = len(basin_results)
if n_basins == 0:
    print("No results to plot")
else:
    n_cols = min(4, n_basins)
    n_rows = (n_basins + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(4*n_cols, 3*n_rows))
    axes = np.atleast_1d(axes).flatten()
    
    for idx, (basin_name, df) in enumerate(basin_results.items()):
        ax = axes[idx]
        
        if df.empty:
            ax.set_title(f"{basin_name}: No data")
            continue
        
        config = get_basin_config(BASINS_TO_ANALYZE[basin_name]["paper_name"])
        ref = get_reference_delta_L(BASINS_TO_ANALYZE[basin_name]["paper_name"])
        
        delta_L = df['delta_L'].dropna()
        stats = compute_asymmetry_statistics(delta_L)
        
        ax.hist(delta_L, bins=30, alpha=0.7, edgecolor='black')
        ax.axvline(stats['median'], color='blue', linestyle='-', linewidth=2, 
                   label=f'Computed: {stats["median"]:.2f}')
        ax.axvline(ref['median'], color='red', linestyle='--', linewidth=2,
                   label=f'Reference: {ref["median"]:.2f}')
        
        ax.set_xlabel('delta_L')
        ax.set_ylabel('Count')
        short_name = config["full_name"].split(',')[0]
        ax.set_title(f"{short_name}\n(n={len(delta_L)}, th={STREAM_THRESHOLD})")
        ax.legend(fontsize=7)
        ax.set_xlim(0, 2.5)
    
    # Hide unused subplots
    for idx in range(len(basin_results), len(axes)):
        axes[idx].set_visible(False)
    
    plt.tight_layout()
    plt.suptitle(f'Experiment: {EXPERIMENT_NAME} (threshold={STREAM_THRESHOLD})', y=1.02, fontsize=14)
    
    # Save figure
    fig.savefig(OUTPUT_DIR / "delta_L_distribution.png", dpi=150, bbox_inches='tight')
    print(f"Saved figure to: {OUTPUT_DIR / 'delta_L_distribution.png'}")
    plt.show()

---
## Export Combined Results

---
## Aridity Index vs. Delta-L

In [ ]:
# Scatter plot: Aridity Index vs. Lengthwise Asymmetry (delta_L)
if len(basin_results) == 0:
    print("No results to plot")
else:
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Collect data for all analyzed basins
    plot_data = []
    for basin_name, df in basin_results.items():
        if df.empty:
            continue
        
        config = get_basin_config(BASINS_TO_ANALYZE[basin_name]["paper_name"])
        ref = get_reference_delta_L(BASINS_TO_ANALYZE[basin_name]["paper_name"])
        stats = compute_asymmetry_statistics(df['delta_L'].dropna())
        
        plot_data.append({
            "name": config['full_name'].split(',')[0],
            "aridity": config["aridity_index"],
            "aridity_25": config["aridity_index_25"],
            "aridity_75": config["aridity_index_75"],
            "computed_median": stats['median'],
            "computed_25": stats['p25'],
            "computed_75": stats['p75'],
            "ref_median": ref['median'],
            "ref_25": ref['p25'],
            "ref_75": ref['p75'],
        })
    
    # Plot computed values (blue circles)
    for i, d in enumerate(plot_data):
        ax.errorbar(
            d["aridity"], d["computed_median"],
            xerr=[[d["aridity"] - d["aridity_25"]], [d["aridity_75"] - d["aridity"]]],
            yerr=[[d["computed_median"] - d["computed_25"]], [d["computed_75"] - d["computed_median"]]],
            fmt='o', markersize=10, capsize=4, capthick=2,
            color='dodgerblue', ecolor='dodgerblue', alpha=0.8,
            label='Computed' if i == 0 else None
        )
    
    # Plot reference values (red squares)
    for i, d in enumerate(plot_data):
        ax.errorbar(
            d["aridity"], d["ref_median"],
            xerr=[[d["aridity"] - d["aridity_25"]], [d["aridity_75"] - d["aridity"]]],
            yerr=[[d["ref_median"] - d["ref_25"]], [d["ref_75"] - d["ref_median"]]],
            fmt='s', markersize=8, capsize=4, capthick=2,
            color='crimson', ecolor='crimson', alpha=0.8,
            label='Paper Reference' if i == 0 else None
        )
    
    # Connect computed and reference with dashed lines
    for d in plot_data:
        ax.plot([d["aridity"], d["aridity"]], 
                [d["computed_median"], d["ref_median"]], 
                'k--', alpha=0.3, linewidth=1)
    
    # Add labels
    for d in plot_data:
        y_pos = max(d["computed_median"], d["ref_median"])
        ax.annotate(d["name"], (d["aridity"], y_pos),
                    textcoords="offset points", xytext=(5, 5), fontsize=8)
    
    ax.set_xlabel('Aridity Index', fontsize=12)
    ax.set_ylabel('delta_L (Lengthwise Asymmetry)', fontsize=12)
    ax.set_title(f'Aridity Index vs. delta_L - {EXPERIMENT_NAME} (th={STREAM_THRESHOLD})\n'
                 f'(Error bars show 25th-75th percentiles)', fontsize=12)
    ax.legend(loc='upper left', fontsize=10)
    ax.grid(True, alpha=0.3)
    ax.set_xlim(0, None)
    ax.set_ylim(0, None)
    
    plt.tight_layout()
    fig.savefig(OUTPUT_DIR / "aridity_vs_delta_L.png", dpi=150, bbox_inches='tight')
    print(f"Saved figure to: {OUTPUT_DIR / 'aridity_vs_delta_L.png'}")
    plt.show()

In [ ]:
# Combine all results
all_results = []
for basin_name, df in basin_results.items():
    if df.empty:
        continue
    df_copy = df.copy()
    df_copy['basin'] = basin_name
    config = get_basin_config(BASINS_TO_ANALYZE[basin_name]["paper_name"])
    df_copy['basin_full_name'] = config['full_name']
    df_copy['stream_threshold'] = STREAM_THRESHOLD
    df_copy['experiment'] = EXPERIMENT_NAME
    all_results.append(df_copy)

if all_results:
    df_all = pd.concat(all_results, ignore_index=True)
    
    combined_path = OUTPUT_DIR / "all_basins_combined.csv"
    df_all.to_csv(combined_path, index=False)
    
    print(f"\nCombined results saved to: {combined_path}")
    print(f"Total pairs: {len(df_all)}")
    print(f"Total touching: {df_all['touching'].sum()}")
    print(f"Basins: {df_all['basin'].nunique()}")
else:
    print("No results to combine")

---
## Experiment Complete

Results have been saved to the experiment output directory. To compare with other experiments:

```python
# Load results from different experiments
import pandas as pd
from channel_heads import get_experiment_output_dir

df_th145 = pd.read_csv(get_experiment_output_dir("th145_baseline") / "all_basins_combined.csv")
df_th500 = pd.read_csv(get_experiment_output_dir("th500_test") / "all_basins_combined.csv")

# Compare delta_L distributions
print(f"Threshold 145: median delta_L = {df_th145['delta_L'].median():.3f}")
print(f"Threshold 500: median delta_L = {df_th500['delta_L'].median():.3f}")
```